# EPA1361 - Model-Based Decision Making
# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

* Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment
  

* Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function. 

* *hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply slice for 1:50 and 1:250.

* Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)

# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging, MultiprocessingEvaluator)

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

from predprey_function import pred_prey

# Model Definition and Experiments

In [3]:
#Initiate Model
model = Model(name="PredPreyModel", function = pred_prey)

#Define Uncertainties
model.uncertainties = [RealParameter('prey_birth_rate', 0.015 , 0.035),
                       RealParameter('predation_rate', 0.0005 , 0.003),
                       RealParameter('predator_efficiency', 0.001 , 0.004),
                       RealParameter('predator_loss_rate', 0.04 , 0.08)]

#Define Outcomes
model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

#Turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

n_scenarios = 100

with MultiprocessingEvaluator(model, n_processes=7) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(n_scenarios)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 70 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 90 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [4]:
experiments

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model
0,0.001458,0.003410,0.063044,0.016700,0,None,PredPreyModel
1,0.002650,0.001126,0.073569,0.032234,1,None,PredPreyModel
2,0.001208,0.002480,0.058983,0.029560,2,None,PredPreyModel
3,0.001156,0.003104,0.068910,0.023272,3,None,PredPreyModel
4,0.001065,0.003841,0.040115,0.016425,4,None,PredPreyModel
...,...,...,...,...,...,...,...
95,0.000733,0.003283,0.069336,0.026507,95,None,PredPreyModel
96,0.002957,0.002304,0.072312,0.018592,96,None,PredPreyModel
97,0.002999,0.001094,0.077885,0.015476,97,None,PredPreyModel
98,0.001442,0.001241,0.045097,0.026731,98,None,PredPreyModel


## Linear regression

In [112]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

experiments_clean = experiments.drop(columns=['scenario','policy','model'])

experiments_clean.values.dtype

dtype('float64')

In [113]:
#create the right dataframe
prey = outcomes['prey']
prey

array([[[ 50.        ,  49.84424891,  49.67922255, ...,   5.58251661,
           5.60311011,   5.62380954]],

       [[ 50.        ,  49.74043706,  49.48506747, ..., 102.29055135,
         102.209556  , 102.11921639]],

       [[ 50.        ,  50.06739175,  50.12995822, ...,  17.96448882,
          17.72052013,  17.48120579]],

       ...,

       [[ 50.        ,  49.44379332,  48.89806845, ..., 108.55588552,
         108.93564435, 109.31631872]],

       [[ 50.        ,  49.97354142,  49.94556847, ...,  51.06069924,
          51.04967186,  51.03703748]],

       [[ 50.        ,  49.78778285,  49.56623238, ...,   7.48664223,
           7.48499683,   7.48400823]]])

In [114]:
prey_mean = np.mean(outcomes['prey'][:,0,:], axis=1)
prey_mean

LR_mean = statsmodels.regression.linear_model.OLS(prey_std, experiments_clean.values)

fitted_mean = LR_mean.fit()

fitted_mean.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.940
Model:                            OLS   Adj. R-squared (uncentered):              0.938
Method:                 Least Squares   F-statistic:                              377.1
Date:                Thu, 14 May 2020   Prob (F-statistic):                    9.12e-58
Time:                        16:15:28   Log-Likelihood:                         -272.22
No. Observations:                 100   AIC:                                      552.4
Df Residuals:                      96   BIC:                                      562.9
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2376.5474    527.678      4.504      0.000    1329.115    3423.980
x2          1300.3749    396.293      3.281      0.001     513.740    2087.010
x3           108.0585     25.686      4.207      0.000      57.071     159.046
x4            19.0589     58.217      0.327      0.744     -96.502     134.619
==============================================================================
Omnibus:                       47.507   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.368
Skew:                           1.491   Prob(JB):                     4.19e-45
Kurtosis:                       9.337   Cond. No.                         92.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
prey_final = outcomes['prey'][:,:,-1].flatten()
prey_final

LR_final = statsmodels.regression.linear_model.OLS(prey_final, experiments_clean.values)

fitted_final = LR_final.fit()


fitted_final.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.786
Model:                            OLS   Adj. R-squared (uncentered):              0.777
Method:                 Least Squares   F-statistic:                              88.22
Date:                Thu, 14 May 2020   Prob (F-statistic):                    2.72e-31
Time:                        16:15:30   Log-Likelihood:                         -428.83
No. Observations:                 100   AIC:                                      865.7
Df Residuals:                      96   BIC:                                      876.1
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1447.9592   2526.482      0.573      0.568   -3567.067    6462.986
x2           -1.2e+04   1897.419     -6.326      0.000   -1.58e+04   -8236.119
x3           854.7270    122.985      6.950      0.000     610.604    1098.850
x4           256.2848    278.740      0.919      0.360    -297.010     809.579
==============================================================================
Omnibus:                        4.746   Durbin-Watson:                   1.877
Prob(Omnibus):                  0.093   Jarque-Bera (JB):                4.806
Skew:                           0.516   Prob(JB):                       0.0905
Kurtosis:                       2.706   Cond. No.                         92.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [116]:
prey_std = np.std(outcomes['prey'][:,0,:], axis=1)
prey_std

LR_std = statsmodels.regression.linear_model.OLS(prey_std, experiments_clean.values)

fitted_std = LR_std.fit()

fitted_std.summary()

#results = smf.ols('prey[[0]] ~ np.log(prey_birth_rate) + np.log(predation_rate)', data=experiments).fit()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.940
Model:                            OLS   Adj. R-squared (uncentered):              0.938
Method:                 Least Squares   F-statistic:                              377.1
Date:                Thu, 14 May 2020   Prob (F-statistic):                    9.12e-58
Time:                        16:15:32   Log-Likelihood:                         -272.22
No. Observations:                 100   AIC:                                      552.4
Df Residuals:                      96   BIC:                                      562.9
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2376.5474    527.678      4.504      0.000    1329.115    3423.980
x2          1300.3749    396.293      3.281      0.001     513.740    2087.010
x3           108.0585     25.686      4.207      0.000      57.071     159.046
x4            19.0589     58.217      0.327      0.744     -96.502     134.619
==============================================================================
Omnibus:                       47.507   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.368
Skew:                           1.491   Prob(JB):                     4.19e-45
Kurtosis:                       9.337   Cond. No.                         92.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""